In [ ]:
import numpy as np
import pandas as pd
# Define activation functions and their derivatives
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def cross_entropy_loss(predictions, targets):
    m = targets.shape[0]
    return -np.sum(targets * np.log(predictions + 1e-9)) / m

# Simulate Data (for testing the NN architecture while ensuring smooth execution)
np.random.seed(42)
num_samples = 100  # Number of images (mocked data)
image_size = (64, 64)  # Resized image dimensions
num_classes = 10  # Number of target classes

# Create mock training data
mock_train_data = np.random.rand(num_samples, image_size[0] * image_size[1])  # Random image data
mock_train_labels = np.eye(num_classes)[np.random.choice(num_classes, num_samples)]  # One-hot encoded labels

# Initialize neural network parameters
input_size = image_size[0] * image_size[1]  # Input dimension
hidden_layer_size = 128  # Hidden layer size
output_size = num_classes  # Number of classes (output layer)

weights_input_hidden = np.random.rand(input_size, hidden_layer_size) * 0.01
weights_hidden_output = np.random.rand(hidden_layer_size, output_size) * 0.01
bias_hidden = np.zeros((1, hidden_layer_size))
bias_output = np.zeros((1, output_size))

# Training parameters
learning_rate = 0.01
num_epochs = 50

# Training loop
for epoch in range(num_epochs):
    # Forward propagation
    hidden_layer_input = np.dot(mock_train_data, weights_input_hidden) + bias_hidden
    hidden_layer_output = sigmoid(hidden_layer_input)
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    predictions = softmax(output_layer_input)

    # Compute loss
    loss = cross_entropy_loss(predictions, mock_train_labels)
    
    # Backward propagation
    output_error = predictions - mock_train_labels  # Gradient of loss w.r.t. output layer
    hidden_error = np.dot(output_error, weights_hidden_output.T) * sigmoid_derivative(hidden_layer_output)

    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, output_error)
    weights_input_hidden -= learning_rate * np.dot(mock_train_data.T, hidden_error)
    bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

    # Log progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{num_epochs} - Loss: {loss:.4f}")

print("Training complete with mock data!")


In [ ]:
# Preprocess images from the second training folder
train_data_2, train_targets_2 = preprocess_images("training set part 2", labels_dict)

# Continue training loop with the second folder data
for epoch in range(num_epochs):
    # Forward propagation
    hidden_layer_input = np.dot(train_data_2, weights_input_hidden) + bias_hidden
    hidden_layer_output = 1 / (1 + np.exp(-hidden_layer_input))  # Sigmoid activation
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    predictions = np.exp(output_layer_input - np.max(output_layer_input, axis=1, keepdims=True))
    predictions /= np.sum(predictions, axis=1, keepdims=True)  # Softmax activation

    # Compute loss
    loss = -np.sum(train_targets_2 * np.log(predictions + 1e-9)) / train_targets_2.shape[0]
    
    # Backward propagation
    output_error = predictions - train_targets_2  # Gradient of loss w.r.t. output layer
    hidden_error = np.dot(output_error, weights_hidden_output.T) * hidden_layer_output * (1 - hidden_layer_output)

    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, output_error)
    weights_input_hidden -= learning_rate * np.dot(train_data_2.T, hidden_error)
    bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

    # Log progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{num_epochs} - Loss: {loss:.4f}")

print("Training complete with the second folder data!")

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Load the CSV file
labels_path = "Train Images Labeling.csv"  # Path to your CSV file
labels_df = pd.read_csv(labels_path)

# Convert to dictionary: {ImageId: ClassName}
labels_dict = dict(zip(labels_df["ImageId"], labels_df["ClassName"]))

# Map class names to numerical labels for one-hot encoding
unique_classes = sorted(set(labels_df["ClassName"]))
class_to_index = {class_name: i for i, class_name in enumerate(unique_classes)}

# Update labels_dict to use numerical labels
labels_dict = {key: class_to_index[value] for key, value in labels_dict.items()}

# Function to preprocess images
def preprocess_images(folder_path, labels_dict, image_size=(64, 64)):
    data = []
    targets = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".jpg"):
            # Load and resize image
            img_path = os.path.join(folder_path, file_name)
            img = Image.open(img_path).convert("L")  # Convert to grayscale
            img_resized = img.resize(image_size)
            img_array = np.array(img_resized).flatten() / 255.0  # Normalize pixel values
            
            # Append to data and get label
            data.append(img_array)
            if file_name in labels_dict:
                label = labels_dict[file_name]
                one_hot_label = np.eye(len(unique_classes))[label]
                targets.append(one_hot_label)
    
    return np.array(data), np.array(targets)

# Path to your training folders
train_folder_1 = "Part 1 - training set"
train_folder_2 = "training set part 2"

# Process training images
train_data_1, train_targets_1 = preprocess_images(train_folder_1, labels_dict)
train_data_2, train_targets_2 = preprocess_images(train_folder_2, labels_dict)

# Combine both training parts
train_data = np.vstack((train_data_1, train_data_2))
train_targets = np.vstack((train_targets_1, train_targets_2))

# Initialize neural network parameters
image_size = (64, 64)  # Resized image dimensions
input_size = image_size[0] * image_size[1]  # Input dimension
hidden_layer_size = 128  # Hidden layer size
output_size = len(unique_classes)  # Number of classes (output layer)

np.random.seed(42)
weights_input_hidden = np.random.rand(input_size, hidden_layer_size) * 0.01
weights_hidden_output = np.random.rand(hidden_layer_size, output_size) * 0.01
bias_hidden = np.zeros((1, hidden_layer_size))
bias_output = np.zeros((1, output_size))

# Training parameters
learning_rate = 0.01
num_epochs = 50

# Training loop
for epoch in range(num_epochs):
    # Forward propagation
    hidden_layer_input = np.dot(train_data, weights_input_hidden) + bias_hidden
    hidden_layer_output = 1 / (1 + np.exp(-hidden_layer_input))  # Sigmoid activation
    output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
    predictions = np.exp(output_layer_input - np.max(output_layer_input, axis=1, keepdims=True))
    predictions /= np.sum(predictions, axis=1, keepdims=True)  # Softmax activation

    # Compute loss
    loss = -np.sum(train_targets * np.log(predictions + 1e-9)) / train_targets.shape[0]
    
    # Backward propagation
    output_error = predictions - train_targets  # Gradient of loss w.r.t. output layer
    hidden_error = np.dot(output_error, weights_hidden_output.T) * hidden_layer_output * (1 - hidden_layer_output)

    # Update weights and biases
    weights_hidden_output -= learning_rate * np.dot(hidden_layer_output.T, output_error)
    weights_input_hidden -= learning_rate * np.dot(train_data.T, hidden_error)
    bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

    # Log progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{num_epochs} - Loss: {loss:.4f}")

print("Training complete!")

# Test set processing
test_folder = "Test_set"
test_data, _ = preprocess_images(test_folder, {}, image_size=image_size)  # No labels for test set

# Test set predictions
hidden_layer_input = np.dot(test_data, weights_input_hidden) + bias_hidden
hidden_layer_output = 1 / (1 + np.exp(-hidden_layer_input))
output_layer_input = np.dot(hidden_layer_output, weights_hidden_output) + bias_output
test_predictions = np.exp(output_layer_input - np.max(output_layer_input, axis=1, keepdims=True))
test_predictions /= np.sum(test_predictions, axis=1, keepdims=True)

# Output predicted classes for the test set
predicted_classes = np.argmax(test_predictions, axis=1)



In [ ]:
labels_path = "Train Images Labeling.csv"
labels_df = pd.read_csv(labels_path)

# Print column names
print(labels_df.columns)